In [29]:
import numpy as np
from math import cos, sin, radians, sqrt, pi, atan, tan


In [36]:
class Elemento():
    def __init__(self,elemento, base, altura, moduloElastico, coordenada_xi, coordenada_yi, coordenada_xf, coordenada_yf, conexion):
        
        self.Elem = elemento
        self.b = base
        self.h = altura
        self.E = moduloElastico
        self.cxi = coordenada_xi
        self.cyi = coordenada_yi
        self.cxf = coordenada_xf
        self.cyf = coordenada_yf
        self.conex = conexion
        # Propiedades del elemento
        self.A = self.Area()
        self.I = self.Inercia()
        self.L = self.Longitud()
        self.ang = self.Angulo()
        self.l = self.coseno()
        self.m = self.seno()

    # Propiedades del elemento

    def Area(self):
        return self.b * self.h

    def Inercia(self):
        return (self.b * self.h**3) / 12

    def Longitud(self):
        return sqrt((self.cxf - self.cxi)**2 + (self.cyf - self.cyi)**2)

    
    def Angulo(self):

        b = (self.cxf - self.cxi)
        h = (self.cyf - self.cyi)
        try:
            angulo = atan(h/b)*180/pi
        except ZeroDivisionError:
            angulo = 90
        return angulo

    def seno(self):
        return sin(radians(self.ang))

    def coseno(self):
        return cos(radians(self.ang))        



In [37]:

E  = 221.359

tbl_Elem = [
    
  # Elem, Ni, Nf, b, h, E,
   ['1', 1, 2, 30, 30, E],
 ['2', 2, 3, 30, 60, E],
 ['3', 4, 3, 30, 30, E],]
    
tbl_Nod = [
    #Nodo, cx, cy
 [1, 0, 0.0],
 [2, 0, 400.0],
 [3, 1000.0, 400],
 [4, 1000, 0.0],

 ]
        #El, Ni, Nf
#print(nodos[:])
tbl_Fuer = [
    
    #Nod, fx, fy, Mz
    [2,100, 0, 0],
  
    ]

tbl_Fuer_W = [
    #Elem, Li,Lf, Wi, Wf
    #["1", 0, 500, -50, 0], # borrar Li y Lf

    ]

tbl_Rest = [
        #Tipo de Apoyo, Nodo
        # Simple
        # Articulado
        # Empotrado
    ["Empotrado", 1],
    ["Articulado", 4,],
    #["Articulado", 4,],
    ]

"""====Diccionarios===="""
# ==============Diccionarios de elementos==================#

diccionarioE = {}

for i in range(len(tbl_Elem)):
    Elem = tbl_Elem[i][0]
    Ni = tbl_Elem[i][1]
    Nf = tbl_Elem[i][2]
    b = tbl_Elem[i][3]
    h = tbl_Elem[i][4]
    E = tbl_Elem[i][5]   
    diccionarioE[Elem] = [Ni, Nf, b, h, E]

# ==============Diccionarios de nodos==================#
    
diccionarioN = {}

for i in range(len(tbl_Nod)):
    Nod = tbl_Nod[i][0]
    cx = tbl_Nod[i][1]
    cy = tbl_Nod[i][2]
    diccionarioN[Nod] = [cx, cy]
    

In [38]:
elementos = [] # Almacena todos los elementos con sus propiedades

def ext_elem(): # Elementos fuera del analisis

    for i in range(len(tbl_Elem)):
        Ni = tbl_Elem[i][1]
        Nf = tbl_Elem[i][2]
        b = tbl_Elem[i][3]
        h = tbl_Elem[i][4]
        Ele = tbl_Elem[i][0]    
        conexion = [Ele, Ni, Nf]
        xi = diccionarioN[Ni][0]
        yi = diccionarioN[Ni][1]
        xf = diccionarioN[Nf][0]
        yf = diccionarioN[Nf][1]     
        barra = Elemento(Ele,b, h, E, xi, yi, xf, yf, conexion)
        elementos.append(barra) # Almacena elemento
    return elementos


elementos = ext_elem()
        

In [39]:
# SUBCORTES A LAS CARGAS DISTRIBUIDAS
sub = 2 #Cantidad de subdiviciones que se realizara al elemento
###############################################################
cheq = [] # Guarda coordenadas para luego chequer si ya existen 
          # y asi no crear otro nodo en la misma posición
Sub_elementos = []
coord = [] # Coordenadas de los nodos
dic_coord_x_elemento =  {}
for i in range(len(tbl_Elem)):
    Elem = tbl_Elem[i][0]
    Elemento = elementos[i]
    L = Elemento.L
    L_unitaria = L/sub
    coord_p = []
    ang = Elemento.ang
    #print(ang)
    
    for j in range(sub+1): # Nodos =  Can_sub + 1
        #i = i + 1
        # Elementos horizontales
        coor_p = []
        if ang == 0: 
            cy = Elemento.cyi
            cx = Elemento.cxi + round(L_unitaria*j,5)
            
            coor_p.append([cx,cy])
            cheq.append([cx,cy])
            
            if cheq.count([cx, cy]) < 2: # Se chequea si la coord ya esta incluida
                coord.append([cx,cy])
            
        # Elementos verticales
        elif ang == 90:
            cx = Elemento.cxi
            cy = Elemento.cyi + round(L_unitaria*j,5)
            # print(cy)
            
            cheq.append([cx,cy])
            coord_p.append([cx,cy])
            if cheq.count([cx, cy]) < 2: # Se chequea si la coord ya esta incluida
                coord.append([cx,cy])
                
        # Elementos inclinados crecientes
        elif ang > 0 and ang < 90:
            """estos fracmento forman l_uni en un triangulo rectangulo"""
            Ly_uni =  L_unitaria* sqrt(1/(1+(1/(tan(radians(ang))**2)))) # Longitud del fracmento en y
            Lx_uni = Ly_uni/tan(radians(ang)) # Longitud del fracmento en x
            cx = Elemento.cxi + round(Lx_uni*j,5)
            cy = Elemento.cyi + round(Ly_uni*j,5)
            
            #print("j=",j, "Ly= ",Ly_uni,"Lx= ",Lx_uni, "cx= ",cx, "cy= ", cy)
            cheq.append([cx,cy])
            coord_p.append([cx,cy])
            if cheq.count([cx, cy]) < 2: # Se chequea si la coord ya esta incluida
                
                coord.append([cx,cy])
                
        # Elementos inclinados decrecientes
        elif ang < 0:
            """estos fracmento forman l_uni en un triangulo rectangulo"""
            Ly_uni =  L_unitaria* sqrt(1/(1+(1/(tan(radians(ang))**2)))) # Longitud del fracmento en y
            Lx_uni = Ly/tan(radians(ang)) # Longitud del fracmento en x
            cx = Elemento.cxi - round(Lx_uni*j,5)
            cy = Elemento.cyi - round(Ly_uni*j,5)
            
            cheq.append([cx,cy])
            coord_p.append([cx,cy])
            if cheq.count([cx, cy]) < 2: # Se chequea si la coord ya esta incluida
                coord.append([cx,cy])
        dic_coord_x_elemento[Elem] = coord_p


In [34]:
dic_coord_x_elemento

{'1': [[0, 0.0], [0, 200.0], [0, 400.0]],
 '2': [],
 '3': [[1000, 0.0], [1000, 200.0], [1000, 400.0]]}

In [7]:
coord

[[0, 0.0],
 [0, 200.0],
 [0, 400.0],
 [500.0, 400.0],
 [1000.0, 400.0],
 [1000, 0.0],
 [1000, 200.0]]

In [8]:
Sub_Nodos = []
cont = 0
diccionario_sub_N = {}

for i in range(len(coord)):
    cont = cont + 1
    cx = coord[i][0]
    cy = coord[i][1]
    Sub_Nodos.append([cont, cx, cy])
    diccionario_sub_N[cont] = [cx, cy]

In [9]:
Sub_Nodos

[[1, 0, 0.0],
 [2, 0, 200.0],
 [3, 0, 400.0],
 [4, 500.0, 400.0],
 [5, 1000.0, 400.0],
 [6, 1000, 0.0],
 [7, 1000, 200.0]]

In [10]:
Sub_Nodos[1][1:]

[0, 200.0]

In [11]:

diccionario_sub_E = {} # Diccionario de elemento subdivididos
cant = 0
Sub_elementos = []
sub_elementos = []
for i in range(len(tbl_Elem)):
    Elem = tbl_Elem[i][0]
    b = tbl_Elem[i][3]
    h = tbl_Elem[i][4]
    E = tbl_Elem[i][5]
    
    Sub_elem_p = [] # Guarda de forma parcial para pasar a dic_sub_elem y se formatea
    coord_sub = dic_coord_x_elemento[Elem]
    for n in range(len(Sub_Nodos)):
        try:
            coord_sub_nod_i = Sub_Nodos[n][1:] # Coordenada del nodo inicial
            coord_sub_nod_f = Sub_Nodos[n+1][1:] # Coordenada del siguiente nodo
            if coord_sub_nod_i in coord_sub:
                if coord_sub_nod_f in coord_sub:
                    cant = cant + 1
                    Ni = Sub_Nodos[n][0]
                    Nf = Sub_Nodos[n+1][0]
                    Sub_elem_p.append([str(cant),Ni, Nf, b, h, E])
                    Sub_elementos.append([str(cant),Ni, Nf, b, h, E])
        except IndexError:
            print("Hi")
    
    diccionario_sub_E[Elem] = Sub_elem_p
    

Hi
Hi
Hi


In [12]:
diccionario_sub_E

{'1': [['1', 1, 2, 30, 30, 221.359], ['2', 2, 3, 30, 30, 221.359]],
 '2': [],
 '3': [['3', 5, 6, 30, 30, 221.359], ['4', 6, 7, 30, 30, 221.359]]}

In [13]:
# Creacion de tablas de sub nodos y sub elementos
tbl_sub_nodos = Sub_Nodos
tbl_sub_elementos = Sub_elementos

In [14]:
tbl_sub_elementos

[['1', 1, 2, 30, 30, 221.359],
 ['2', 2, 3, 30, 30, 221.359],
 ['3', 5, 6, 30, 30, 221.359],
 ['4', 6, 7, 30, 30, 221.359]]

In [15]:
diccionario_sub_N

{1: [0, 0.0],
 2: [0, 200.0],
 3: [0, 400.0],
 4: [500.0, 400.0],
 5: [1000.0, 400.0],
 6: [1000, 0.0],
 7: [1000, 200.0]}

In [18]:
""" Analisis primario de todos los sub elementos """

def ext_sub_elem(): # Elementos antes de realizar el del analisis
    sub_elementos = []
    sub_GDL = []
    #########---------->
    #Grados de libertad de los nodos
    #########---------->
    sub_GDL_nodos = {}
    for i in range(len(tbl_sub_nodos)):
        nodo = tbl_sub_nodos[i][0]
        gdl = [nodo*3-3, nodo*3-2, nodo*3-1]
        sub_GDL_nodos[nodo] = gdl   

    for i in range(len(tbl_sub_elementos)):
        b = tbl_sub_elementos[i][3]
        h = tbl_sub_elementos[i][4]
            
        Elem = tbl_sub_elementos[i][0]
        Ni = tbl_sub_elementos[i][1]
        Nf = tbl_sub_elementos[i][2]
        sub_conexion = [Elem, Ni, Nf]
        xi = diccionario_sub_N[Ni][0]
        yi = diccionario_sub_N[Ni][1]
        xf = diccionario_sub_N[Nf][0]
        yf = diccionario_sub_N[Nf][1]
            
        sub_barra = Elemento(Elem,b, h, E, xi, yi, xf, yf, sub_conexion)
        sub_elementos.append(sub_barra) # Almacena elemento
    return sub_elementos

SUB_Elementos = ext_sub_elem() # Almacena todos los elementos con sus propiedades

In [19]:
SUB_Elementos

In [20]:
SUB_Elementos[9].ang

IndexError: list index out of range

In [21]:
# Obtener cargar externas para los sub elementos
coord = []

tbl_sub_F_W = []

for i in range(len(tbl_Fuer_W)):
    Elemento = tbl_Fuer_W[i][0]
    wi = tbl_Fuer_W[i][3]
    wf = tbl_Fuer_W[i][4]
    L = 0
    
    for n in range(len(elementos)):
        if Elemento == elementos[n].Elem:
            L = elementos[n].L
            
    for j in range(sub):
        Elem = diccionario_sub_E[Elemento][j][0]
        ################################################
        ##################################################
        
        # Carga distibuida rectangular
        #print(wi, wf)
        if wi == wf:
            
            L_unitaria = L/sub*(j+1)
            #print(L_unitaria)
            sub_W = wi
            tbl_sub_F_W.append([Elem, L/sub*j, L_unitaria,sub_W , sub_W])
            
            
        #########################################################
        ##############################################################
            
        # Carga distibuida triangular decreciente
        
        if  abs(wi) > 0  and abs(wf) == 0:
            
            W = wi
            L_unitaria = L/sub
            sub_wi = (W * L_unitaria * j) /L 
            sub_wf = (W * L_unitaria * (j+1))/L
            tbl_sub_F_W.append([Elem, 0, L_unitaria,sub_wi , sub_wf])
        
        
        # Carga distibuida triangular creciente
        if  abs(wf) > 0  and abs(wi) == 0:
            
            W = wf
            L_unitaria = L/sub
            sub_wi = (W * L_unitaria * j) /L 
            sub_wf = (W * L_unitaria * (j+1))/L
            tbl_sub_F_W.append([Elem, 0, L_unitaria,sub_wi , sub_wf])
        
        # Carga distibuida trapezoidal decreciente
        if  abs(wi) > abs(wf) and abs(wf) > 0:
            
            W = (wi - wf)
            L_unitaria = L/sub
            sub_wi = (W * L_unitaria * j) /L + wf
            sub_wf = (W * L_unitaria * (j+1))/L + wf
            tbl_sub_F_W.append([Elem, 0, L_unitaria,sub_wi , sub_wf])
        
        # Carga distibuida trapezoidal creciente
        if  abs(wi) < abs(wf) and abs(wi) > 0:
            
            W = (wf - wi)
            L_unitaria = L/sub
            sub_wi = wi + (W * L_unitaria * j) /L
            sub_wf = wi + (W * L_unitaria * (j+1))/L
            tbl_sub_F_W.append([Elem, 0, L_unitaria,sub_wi , sub_wf])


In [22]:
tbl_sub_elementos

[['1', 1, 2, 30, 30, 221.359],
 ['2', 2, 3, 30, 30, 221.359],
 ['3', 5, 6, 30, 30, 221.359],
 ['4', 6, 7, 30, 30, 221.359]]

In [23]:
tbl_sub_nodos

[[1, 0, 0.0],
 [2, 0, 200.0],
 [3, 0, 400.0],
 [4, 500.0, 400.0],
 [5, 1000.0, 400.0],
 [6, 1000, 0.0],
 [7, 1000, 200.0]]

In [24]:
tbl_sub_F_W

[]